<a href="https://colab.research.google.com/github/rkiee/project-emr2icd/blob/main/BERT_EMR%EA%B8%B0%EB%B0%98_%EC%A7%84%EB%8B%A8%EC%B6%94%EC%B2%9C%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install transformers

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
file_path = '/content/신장내과_본원_외래_20230701_20231231_filtered.xlsx'

selected_columns = ['ICD10', 'transstr']
df = pd.read_excel(file_path, usecols=selected_columns)

print('전체 데이터 수 :',len(df))
# df.head(5)

전체 데이터 수 : 826


In [3]:
# 학습용, 테스트용, 검증용 데이터 분할
X_train, X_temp, y_train, y_temp = train_test_split(df['transstr'], df['ICD10'], test_size=0.2, random_state=5)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=7)

# print(X_train)
# print("=======================================\n")
# print(X_test)
# print("=======================================\n")
# print(X_val)
# print("=======================================\n")
# print(y_train)
# print("=======================================\n")
# print(y_test)
# print("=======================================\n")
# print(y_val)

In [34]:
#https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/

In [ ]:
# device_name = "cuda" if torch.cuda.is_available() else "cpu"
# device = torch.device(device_name)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained('NitzanBar/umls-spanbert')

# Model
model = AutoModelForSequenceClassification.from_pretrained('NitzanBar/umls-spanbert')
# model.to(device)

In [5]:
print(y_train[:20])

443    N185
441    N183
66     R311
773    N183
131    N183
785    R601
356    N184
208    N178
346    N391
540    N184
296    R601
614    N185
306    N182
279    N183
266    N028
153    Z524
163    Z524
353    R311
45     Z524
366    N391
Name: ICD10, dtype: object


In [6]:
# label encoding
unique_label = y_train.unique() # 중복없는 고유한 값 추출
print("1 :", unique_label)

label_encoder = LabelEncoder() # 데이터를 숫자 형태로 인코딩할 수 있도록 함
label_encoder.fit(unique_label)
# LabelEncoder 객체가 실제 데이터에 맞게 학습 (레이블과 해당하는 숫자 값 간의 매핑을 학습)
# LabelEncoder 객체가 학습된 후에는 어떤 레이블이 주어져도 그에 해당하는 숫자 값으로 변환할 수 있다

# print("2 : ", label_encoder)

1 : ['N185' 'N183' 'R311' 'R601' 'N184' 'N178' 'N391' 'N182' 'N028' 'Z524'
 'R808' 'N029' 'N059' 'R798']


LabelEncoder()

In [27]:
temp_df = y_train.to_frame()
temp_df['encoded_ICD10'] = label_encoder.transform(temp_df['ICD10']) # 숫자로 변환 : 머신 러닝 모델은 보통 숫자를 입력으로 사용하므로
y_train_encoding = pd.Series(temp_df['encoded_ICD10'])
# Y_train_encoding = Y_train_encoding.rename('encoded_ICD10')
print("2 :", y_train_encoding[:20])

# import numpy as np
# min_value = np.min(y_train_encoding)
# max_value = np.max(y_train_encoding)
# print("3 :", min_value, max_value)

2 : 443     7
441     5
66      9
773     5
131     5
785    10
356     6
208     3
346     8
540     6
296    10
614     7
306     4
279     5
266     0
153    13
163    13
353     9
45     13
366     8
Name: encoded_ICD10, dtype: int64
3 : 0 13


In [8]:
print(temp_df)

    ICD10  encoded_ICD10
443  N185              7
441  N183              5
66   R311              9
773  N183              5
131  N183              5
..    ...            ...
73   Z524             13
400  R311              9
118  N183              5
701  N184              6
206  N183              5

[660 rows x 2 columns]


In [9]:
# print(y_train_encoding[:20])
print(y_train_encoding)

443     7
441     5
66      9
773     5
131     5
       ..
73     13
400     9
118     5
701     6
206     5
Name: encoded_ICD10, Length: 660, dtype: int64


In [28]:
class EMRDiagcdDataset(Dataset):

    def __init__(self, df):
        self.df = df
        # print(len(self.df))

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        record = self.df.iloc[index]['transstr']
        # diagcd = self.df.iloc[index]['ICD10']
        diagcd = self.df.iloc[index]['encoded_ICD10']
        # print(f"diagcd : {diagcd} , index : {index}")

        return record, diagcd

        # tokens = self.tokenizer(
        #     record,
        #     return_tensors='pt',
        #     truncation=True,
        #     padding='max_length')

        # input_ids = tokens[input_ids].squeeze(0)
        # attention_mask  = tokens[attention_mask ].squeeze(0)
        # token_type_ids  = torch.zeros_like(token_type_ids )

        # return {
        #     'input_ids' : input_ids,
        #     'attention_mask' : attention_mask,
        #     'token_type_ids' : token_type_ids,
        # }, torch.tensor(diagcd)

In [35]:
train_df = pd.concat([X_train, y_train_encoding, y_train], axis=1)
train_dataset = EMRDiagcdDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [36]:
train_dataset.__getitem__(0)

('2021.09 CABG surgery 2023.06 CR 4.0, EGFR 14 2023.09 CR 5.4, EGFR 11, K 6.5 Seoul Samsung Hospital professor said that',
 7)

In [37]:
print(train_df[:16])

                                              transstr  encoded_ICD10 ICD10
443  2021.09 CABG surgery 2023.06 CR 4.0, EGFR 14 2...              7  N185
441  Since 2016, Hematuria had hematuria in 2022 TY...              5  N183
66   23.04 LMC obstetrics and gynecology visits Hem...              9  R311
773  60s HTN Diagnosis 2023.11.21 CR 1.58 BP 121/67...              5  N183
131  I didn't have a previous checkup.2023. CR 1.83...              5  N183
785  It has been edema since childhood, and 2023.11...             10  R601
356  # HTN (20Y)2023. CR 2.26 EGFR 20.66 HBA1C 7.3 ...              6  N184
208  Ginseng and antler have been taking two years ...              3  N178
346  In high school examination, Hematururia, prote...              8  N391
540  2019. CR 1.11 EGFR 63 2022. CR 1.23 EGFR 54 20...              6  N184
296  10 years ago, I was diagnosed with renal cyst ...             10  R601
614  Hemodialysis was held from 2023.09.27, and it ...              7  N185
306  I visit

In [38]:
train_df.to_excel('/content/train_df.xlsx')

In [39]:
# Adam 옵티마이저 초기화
# Adam : 신경망 훈련하는데 사용되는 최적화 알고리즘
optimizer = Adam(model.parameters(), lr=1e-2)
# 최적화될 모델의 매개변수(가중치와 편향)를 제공, 학습률(0.01(과학적 표기법에서 1e-2)) 설정
criterion = torch.nn.Softmax() # Softmax 활성화 함수 생성
itr = 1 # 반복 횟수
p_itr = 50
epochs = 1
total_loss = 0 # 훈련 중 계산된 손실 저장 변수
total_len = 0 # 훈련 중 처리된 전체 샘플 수 저장 변수
total_correct = 0 # 분류 작업에서 훈련 중 올바르게 예측된 전체 샘플 수를 저장하는 변수

In [ ]:
print(f"Number of output classes: BertConfig: {model.config}")

In [ ]:
# Print the number of output classes in the model configuration
print(f"Number of output classes: {model.config.num_labels}")

In [42]:
# train
model.train() # 모델을 훈련 모드로 설정

i = 1

for record, diagcd in train_loader:
    optimizer.zero_grad() # optimizer의 gradients를 초기화
    # print("record: ", record)
    # print("diagcd: ", diagcd)
    encoded_record = tokenizer(record, padding=True, truncation=True, max_length=512, return_tensors="pt")
    # encoded_record = encoded_record.to(device)
    # print(type(encoded_record))
    diagcd = torch.tensor(diagcd)
    # diagcd = torch.tensor(list(diagcd), dim=0).to(device)

    outputs = model(**encoded_record, labels=diagcd)

    loss = outputs.loss
    logits = outputs.logits

    predict = torch.argmax(F.softmax(logits), dim=14)
    correct = predict.eq(diagcd)

    total_correct += correct.sum().item()
    total_len += len(diagcd)
    total_loss += loss.item()

    loss.backward()
    optimizer.step()

    if itr % p_itr == 0:
        print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
        total_loss = 0
        total_len = 0
        total_correct = 0

    itr += 1

    print(f"i : {i}")
    i = i + 1
    if (i==2) :
        break

<ipython-input-42-9daaec74e2db>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  diagcd = torch.tensor(diagcd)


IndexError: Target 7 is out of bounds.

In [51]:
# Print the first few elements of 'diagcd' and 'record'
print(f"First few labels: {diagcd[:5]}")
print(f"First few records: {record[:5]}")

First few labels: tensor([11,  9,  5,  7,  9])
First few records: ('The patient is a patient of 63 years old, currently being treated with the diagnostic name [unruptured aneurysm, basilar].# R/O LT.Va Dissection 2023.07.10.DSA W/U was performed herein and was diagnosed with the above findings and will be implemented in this department as a follow -up test, 07.10.LAB phase Bun/CR 28.2/1.24 (MDRD EGFR: 58.9) and confirm it and please contact us for the contrast agent CT.Thank you ns prof4 ======================================================= 143/81-67 3-4 years ago, I tested that the urine test was also okay.', 'A patient who takes various medicines but does not know about the diagnosis name, and is commissioned by the medical treatment of the Microscopic Hematuria.2022.03.08 CR 1.0 EGFR 58 Proteinuria (-) 2023.01.20 CR 0.9 EGFR 65 Proteinuria (-) Urinary SX', '157-89-72 High blood pressure and no hepatitis for 30 years (Viriad) Gout 10 years in elementary school-diagnosed with kidney

In [ ]:
# train
model.train() # 모델을 훈련 모드로 설정

for epoch in range(epochs):
    for record, diagcd in train_loader:
        optimizer.zero_grad() # optimizer의 gradients를 초기화
        # print("record: ", record)
        # print("diagcd: ", diagcd)
        encoded_record = tokenizer(record, padding=True, truncation=True, max_length=512, return_tensors="pt")
        # encoded_record = encoded_record.to(device)
        diagcd = diagcd.clone().detach()
        # print(type(encoded_record))
        # diagcd = torch.tensor(diagcd)
        # diagcd = torch.tensor(list(diagcd), dim=0).to(device)

        outputs = model(**encoded_record, labels=diagcd)

        loss = outputs.loss # 모델의 출력 손실
        logits = outputs.logits # 모델의 출력 로짓(확률을 계산하기 전의 값)

        predict = torch.argmax(F.softmax(logits), dim=14)
        correct = predict.eq(diagcd)

        total_correct += correct.sum().item()
        total_len += len(diagcd)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        if itr % p_itr == 0:
            # print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr += 1

IndentationError: unexpected indent (<ipython-input-23-4d0f789a0c84>, line 5)

In [ ]:
test_df = pd.concat([X_test, y_test], axis=1)
print(test_df)

test_dataset = EMRDiagcdDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True)

In [ ]:
# evaluation
model.eval()

total_len = 0
total_correct = 0

for record, diagcd in test_loader:

    encoded_record = tokenizer(record, padding=True, truncation=True, max_length=512, return_tensors="pt")
    encoded_record, diagcd = encoded_record.to(device), diagcd.to(device)

    outputs = model(**encoded_record, labels=diagcd)

    logits = outputs.logits

    predict = torch.argmax(F.softmax(logits), dim=14)
    correct = predict.eq(diagcd)

    total_correct += correct.sum().item()
    total_len += len(diagcd)

print('Test accuracy: ', total_correct / total_len)